In [1]:
def f(x,y):
    return x - y

def training_set_generator(f,n):
    '''
    Generates a training set from a function with domain [-10, 10] x [-10, 10]
    f - function to generate data
    n - number of points to generate
    '''
    import random
    inputs = []
    outputs = []
    for i in range(n):
        #x = (20*random.random() - 10,20*random.random() - 10)
        x = np.array([[20*random.random() - 10],[20*random.random() - 10]])
        inputs.append(x)
        outputs.append(f(x[0,0], x[1,0]))
    return(inputs,outputs)


In [2]:
import numpy as np   
class Neural_Network():
    def __init__(self):
        
        self.weights = [
            np.matrix(2*np.random.random((3,2)) - 1),
            np.matrix(2*np.random.random((1,3)) - 1)
        ]
        
    def sigmoid(self, x):
        """
        Takes in weighted sum of the inputs and normalizes
        them through between 0 and 1 through a sigmoid function
        """
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """
        The derivative of the sigmoid function used to
        calculate necessary weight adjustments
        """
        return self.sigmoid(x) * (1 - self.sigmoid(x))
        
    def think(self, inputs):
        """
        Pass inputs through the neural network to get output
        input: 2 x 1 array or matrix
        output: 1 x 1 matrix
        """
        #vectorize the sigmoid function so that it can be applied to matrices
        sigmoid = np.vectorize(self.sigmoid)
        
        inputs = np.matrix(inputs.astype(float))
        preactivation = []
        outputs = []
        preactivation.append(self.weights[0] * inputs)
        outputs.append(sigmoid(preactivation[0]))
        outputs.append(self.weights[1] * outputs[0])
        return outputs, preactivation
        
    def train(self, training_inputs, training_outputs, num_epochs):
        """
        We train the model through trial and error, adjusting the
        synaptic weights each time to get a better result
        
        training_inputs = list of 2 x 1 arrays
        training_outputs = list of numbers
        num_epochs = number of times to run through the complete set of inputs
        """
        for epoch in range(num_epochs):
        
            for x,y in zip(training_inputs, training_outputs):
                # Pass training set through the neural network
                outputs, preactivation = self.think(x)

                # Calculate the error rate
                error = y - outputs[1][0,0]

                # Multiply error by input and gradient of the sigmoid function
                # Less confident weights are adjusted more through the nature of the function
                adjustments = [] 
            
                adjustments.append(0.01 * 2 * error * np.transpose(outputs[0]))
                matrix = np.matrix([[0,0],[0,0],[0,0]], dtype = float)
                for i in range(3):
                    for j in range(2):
                        matrix[i,j] = x[j,0] * self.sigmoid_derivative(preactivation[0][i,0]) * self.weights[1][0,i] 
                adjustments.append(0.01 * 2 * error * matrix)
                
                #adjustments = np.dot(x.T, error * self.sigmoid_derivative(output))
    
                # Adjust synaptic weights
                self.weights[1] += adjustments[0]
                self.weights[0] += adjustments[1]
            print(f'epoch {epoch + 1} complete')
            print(f'MSE: {self.evaluate(training_inputs, training_outputs)}')
    def evaluate(self, test_inputs, test_outputs):
        '''
        Calculate the mean squared error
        test_inputs = list of 2 x 1 arrays
        test_outputs = list of numbers
        '''
        error = 0
        for x,y in zip(test_inputs, test_outputs):
            error += (y - self.think(x)[0][1][0,0])**2
        error /= len(test_inputs)
        return error

In [3]:
nn = Neural_Network()

nn.weights

[matrix([[-0.17482664, -0.82841739],
         [ 0.9135875 ,  0.02715389],
         [ 0.11852933,  0.34645425]]),
 matrix([[-0.06177072,  0.14992521, -0.89328363]])]

In [17]:
training_inputs, training_outputs = training_set_generator(f, 10000)

nn.train(training_inputs, training_outputs, 10) 

epoch 1 complete
MSE: 11.419167270498603
epoch 2 complete
MSE: 11.36492736790554
epoch 3 complete
MSE: 11.483663339616914
epoch 4 complete
MSE: 11.144418275547528
epoch 5 complete
MSE: 10.795620809345431
epoch 6 complete
MSE: 12.672015751117195
epoch 7 complete
MSE: 10.826003964642778
epoch 8 complete
MSE: 11.019313038479519
epoch 9 complete
MSE: 11.72438434366318
epoch 10 complete
MSE: 11.129074279889101


In [4]:
nn.think(np.array([[1],[10]]))[0][1][0,0]

-0.7543204515467413